In [1]:
import networkx as nx
import random
import math
import heapq
import queue
import matplotlib.pyplot as plt
import time
%matplotlib inline

# Build a graph

In [2]:
G = nx.MultiDiGraph()
random.seed(1)
IS_DEBUG_MODE = False

1) Add nodes

Add source nodes:

- (Randomly) generate total supply ('total_supply') and unused supply ('unused_supply') for each source node;

- Record all the total supply from all source nodes in 'grand_total_supply';

- The 'demand' attribute is used for NetworkX's built-in min-cost flow algorithms. Negative demand means supply.

In [3]:
num_of_source = 3
grand_total_supply = 0
rand_supply = [0] * num_of_source
for i in range(num_of_source):
    rand_supply[i] = random.randint(1, 10)
for i in range(num_of_source):
    G.add_node('s' + str(i), demand = -rand_supply[i], total_supply = rand_supply[i], unused_supply = rand_supply[i])
    grand_total_supply += rand_supply[i]

Add sink nodes:

- (Randomly) generate prior probabilities ('prior_prob'), and probabilies of detection ('detect_prob') for each sink.

In [4]:
num_of_sink = 6
rand_prior_prob = [0] * num_of_sink
rand_detect_prob = [0] * num_of_sink
for i in range(num_of_sink):
    rand_prior_prob[i] = random.uniform(0.01, 0.99)
sum_rand_prior_prob = sum(rand_prior_prob)
rand_prior_prob = [a / sum_rand_prior_prob for a in rand_prior_prob]
for i in range(num_of_sink):
    rand_detect_prob[i] = random.uniform(0.01, 0.99)
for i in range(num_of_sink):
    G.add_node('t' + str(i), prior_prob = rand_prior_prob[i], detect_prob = rand_detect_prob[i])

Add global sink node

In [5]:
G.add_node('gt', demand = grand_total_supply)

2) Add arcs

(Randomly) add arcs from sources to sinks:

- Set flow = 0.

In [6]:
num_of_arc = 0  # Record number of arcs between sources and sinks 
connectivity_info = []

for i in range(num_of_source):
    for j in range(num_of_sink):
        if random.random() > 0.5:
            G.add_edge('s' + str(i), 't' + str(j), key = 0, flow = 0, weight = 0, capacity = G.node['s'+str(i)]['total_supply'])
            num_of_arc = num_of_arc + 1
            connectivity_info.append([i, j])

# If a source is not connected to any sink, randomly connect it to some sink
for i in range(num_of_source):
    if not G.neighbors('s' + str(i)):
        sink = random.randint(0, num_of_sink - 1)
        G.add_edge('s' + str(i), 't' + str(sink), key = 0, flow = 0, weight = 0, capacity = G.node['s'+str(i)]['total_supply'])
        num_of_arc = num_of_arc + 1
        connectivity_info.append([i, sink])
        
# If a sink is not connected to any source, randomly connect it to some source
for i in range(num_of_sink):
    if not G.predecessors('t' + str(i)):
        source = random.randint(0, num_of_source - 1)
        G.add_edge('s' + str(source), 't' + str(i), key = 0, flow = 0, weight = 0, capacity = G.node['s'+str(source)]['total_supply'])
        num_of_arc = num_of_arc + 1
        connectivity_info.append([source, i])

Add arcs from sinks to the global sink: 

- Set key = 1 (i.e., the 1st time to search this location); 

- Set flow = 0; 

- Set capacity = 1 (for NetworkX's built-in min-cost flow algorithms);

- Compute and set weight (note: the negative of the probability).

In [7]:
for i in range(num_of_sink):
    sink = 't'+str(i)
    w0 = -G.node[sink]['prior_prob'] * G.node[sink]['detect_prob']
    for j in range(grand_total_supply):
        G.add_edge(sink, 'gt', key = j+1, flow = 0, weight = w0, capacity = 1)
        w0 = w0 * (1 - G.node[sink]['detect_prob'])

3) Plot the graph

In [8]:
if IS_DEBUG_MODE:
    nodes_pos_dict = {}
    for i in range(num_of_source):
        nodes_pos_dict['s'+str(i)] = (0, -(i - (num_of_source-1) / 2))
    for i in range(num_of_sink):
        nodes_pos_dict['t'+str(i)] = (num_of_sink / 2, -(i - (num_of_sink-1) / 2))
    nodes_pos_dict['gt'] = (num_of_sink, 0)
    # Setting: figure dimensions
    plt.rcParams['figure.figsize'] = (10, 6)
    nx.draw_networkx(G, pos = nodes_pos_dict)

4) List all nodes and arcs with properties

In [9]:
if IS_DEBUG_MODE:
    print(grand_total_supply)

In [10]:
if IS_DEBUG_MODE:
    for node in G.nodes(data=True):
        print(node)

In [11]:
if IS_DEBUG_MODE:
    for edge in G.edges(data=True):
        print(edge)

In [12]:
print(' '.join(map(str, [num_of_source, num_of_sink, num_of_arc])))
for i in range(num_of_source):
    print(G.node['s'+str(i)]['total_supply'])
for i in range(num_of_sink):
    print(G.node['t'+str(i)]['prior_prob'], end=' ')
    print(G.node['t'+str(i)]['detect_prob'])
for row in connectivity_info:
    print(' '.join(map(str,row)))

3 6 8
3
10
2
0.09489644666221526 0.037780526991566185
0.18088286777558027 0.8290498018414723
0.16444726107147423 0.4341117265469523
0.23674540360309848 0.7570344808087831
0.28580120790127983 0.01206393228408848
0.03722681298635199 0.4464794501737054
0 0
0 2
0 3
1 0
1 1
2 3
1 4
0 5


# New algorithm

## Subroutine for finding an augmenting path

If can find an augmenting path, return an empty set; Else, return the isolated group.

In [13]:
def assign_extra_demand(nodeName):
    # Initialization
    visited_set = set() # Record visited nodes
    Q = queue.Queue()
    Q.put(nodeName)
    in_Q_set = set() # Record nodes currently in Q (since Python has no queue.contains())
    in_Q_set.add(nodeName)
    pred = {} # Record predecessors for building augmenting path
    for i in range(num_of_source):
        pred['s'+str(i)] = None
    for i in range(num_of_sink):
        pred['t'+str(i)] = None
    # Loop
    while Q.empty() is False:
        nd = Q.get()
        in_Q_set.remove(nd)
        visited_set.add(nd)
        # If the node is a sink
        if nd[0] == 't':
            for source in G.predecessors(nd):
                if source not in in_Q_set and source not in visited_set and source not in elim_set:
                    Q.put(source)
                    in_Q_set.add(source)
                    pred[source] = nd
        # If the node is a source, but it has no unused supply
        elif nd[0] == 's' and G.node[nd]['unused_supply'] == 0:
            for sink in G.neighbors(nd):
                if G.edge[nd][sink][0]['flow'] > 0 and sink not in in_Q_set and sink not in visited_set and sink not in elim_set:
                    Q.put(sink)
                    in_Q_set.add(sink)
                    pred[sink] = nd
        # If the node is a source, and it has unused supply
        else: #(nd[0] == 's' and G.node[nd]['unused_supply'] > 0):
            # Decrement unused_supply of the source by 1
            G.node[nd]['unused_supply'] -= 1 
            # Recursively build augmenting path
            cur = nd
            while True:
                pre = pred[cur]
                if pre == None:
                    break
                # For source
                if cur[0] == 's':
                    #old_flow = G.edge[cur][pre][0]['flow']
                    #G.add_edge(cur, pre, key = 0, flow = old_flow + 1)
                    G.edge[cur][pre][0]['flow'] += 1
                # For sink
                else:
                    #old_flow = G.edge[pre][cur][0]['flow']
                    #G.add_edge(pre, cur, key = 0, flow = old_flow - 1)
                    G.edge[pre][cur][0]['flow'] -= 1
                cur = pre
            # There is no node to be eliminated! And remember to break the outer loop!
            visited_set = set()
            break
    return visited_set

## Implement the algorithm

Start the timer:

In [14]:
start_time_our_algo = time.time()

Initialize the algorithm:  

- Build a min heap ('heap_of_weights') and put initial weights into it;

- Use a set ('elim_set') to track which nodes have been eliminated already;

- Record the final demand at each sink in 'final_sink_demand'.

In [15]:
heap_of_weights = []
for i in range(num_of_sink):
    # Each element in heap_of_weights is of the form: {weight, which_sink, search_order(key)}
    heapq.heappush(heap_of_weights, (G.edge['t'+str(i)]['gt'][1]['weight'], 't'+str(i), 1))
elim_set = set()
final_sink_demand = {}
for i in range(num_of_sink):
    final_sink_demand['t'+str(i)] = 0

Main body of the algorithm

In [16]:
while grand_total_supply > 0 and len(elim_set) != num_of_source + num_of_sink:
    top_element_heap = heapq.heappop(heap_of_weights)
    sink = top_element_heap[1]
    if sink in elim_set:
        continue
    potential_elim_set = assign_extra_demand(sink)
    if len(potential_elim_set) == 0:
        #if IS_DEBUG_MODE:
        #    print(sink, "{0:.4f}".format(-top_element_heap[0]))
        # Reduce grand total supply from all sources by one
        grand_total_supply -= 1
        # Increase the final demand of the sink by one
        final_sink_demand[sink] += 1
        # Set y_{kj} = 1
        old_order = top_element_heap[2]
        old_weight = top_element_heap[0]
        #G.add_edge(sink, 'gt', key = old_order, flow = 1, weight = old_weight, capacity = 1)
        G.edge[sink]['gt'][old_order]['flow'] = 1
        # Compute the next weight, and add the edge to the graph
        new_weight = old_weight * (1 - G.node[sink]['detect_prob'])
        new_order = old_order + 1
        G.add_edge(sink, 'gt', key = new_order, flow = 0, weight = new_weight, capacity = 1)
        # Put the weight into the heap
        heapq.heappush(heap_of_weights, (new_weight, sink, new_order))
    else:
        #if IS_DEBUG_MODE:
        #    print(sink, "{0:.4f}".format(-top_element_heap[0]), "-> Failed and found an isolated group! :", potential_elim_set)
        elim_set = elim_set.union(potential_elim_set)
    #if IS_DEBUG_MODE:
    #    edge_list = [edge for edge in G.edges(data=True) if edge[0][0] == 's' and edge[2]['flow'] > 0]
    #    list.sort(edge_list)
    #    print(edge_list)

Show runtime:

In [17]:
time_elapsed_our_algo = time.time() - start_time_our_algo
print("--- %s seconds ---" % time_elapsed_our_algo)

--- 0.19276213645935059 seconds ---


## Output the results

Flows from sources to sinks:

In [18]:
if IS_DEBUG_MODE:
    edge_list = [edge for edge in G.edges(data=True) if edge[0][0] == 's' and edge[2]['flow'] > 0]
    list.sort(edge_list)
    for edge in edge_list:
        print(edge)

Flows from sinks to global sink:

In [19]:
edge_list_2 = [edge for edge in G.edges(data=True) if edge[0][0] == 't' and edge[2]['flow'] > 0]
if IS_DEBUG_MODE:
    for edge in edge_list_2:
        print(edge)

Show optimal value found by our algorithm:

In [20]:
flow_cost_our_algo = 0
for edge in edge_list_2:
    flow_cost_our_algo += edge[2]['weight']
print(flow_cost_our_algo)

-0.561294539223331


Demand at each sink:

In [21]:
if IS_DEBUG_MODE:
    demand_list = [(k,v) for k,v in final_sink_demand.items() if v > 0]
    list.sort(demand_list)
    print(demand_list)

Supply at each source:

In [22]:
if IS_DEBUG_MODE:
    supply_list = [node for node in G.nodes(data=True) if node[0][0] == 's']
    list.sort(supply_list)
    supply_list

# NetworkX's built-in algorithm: capacity_scaling

Start the timer:

In [23]:
start_time_capacity_scaling = time.time()

In [24]:
flow_cost_capacity_scaling, flow_dict_capacity_scaling = nx.capacity_scaling(G)

Show runtime:

In [25]:
time_elapsed_capacity_scaling = time.time() - start_time_capacity_scaling
print("--- %s seconds ---" % time_elapsed_capacity_scaling)

--- 0.11660194396972656 seconds ---


Show optimal value found by the capacity scaling algorithm:

In [26]:
print(flow_cost_capacity_scaling)

-0.561294539223331
